# **Speech Translation Fon vers Français avec Wav2Vec2 et MBart**

---

## **Description du notebook**

Ce notebook implémente un modèle de traduction vocale de bout en bout (End-to-End) pour traduire la langue **Fon** vers le **français**. Il utilise l'architecture suivante :

*   **Encodeur Audio :** [Wav2Vec2-Base](https://huggingface.co/facebook/wav2vec2-base) pré-entraîné de Facebook (gelé pendant l'entraînement).
*   **Adapter Audio :** Un adapter pour adapter les représentations de Wav2Vec2 au décodeur.
*   **Décodeur de Traduction :** [MBart-Large-CC25](https://huggingface.co/facebook/mbart-large-cc25) pré-entraîné de Facebook (partiellement gelé, les 6 premières couches sont gelées).
*   **Classe Personnalisée** `SpeechTranslationModel` :


Le modèle a été fine-tuné sur un dataset de traduction vocale Fon-Fr FFTSC 2025 fourni dans le cadre de l'IWSTL 2025


**Ce notebook vous permet de :**

*   **Explorer et d'éxecuter le code d'entraînement**
*   **Évaluer le modèle** sur un dataset de validation.
*   **Tester la traduction vocale** sur vos propres fichiers audio Fon.



# Chargement des données

In [ ]:
# Chargement de mon Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Install requirements
%%capture
!pip install -q transformers datasets sacremoses sacrebleu

# Préparation des données

**Lignes directives :**
 - Configuration du path
 - Chargement des DF
 - Exploration et analyse des DF

In [ ]:
MODE_DEBUG = False # Pour desactiver ou activer certaines parties du code

In [ ]:
import os
import numpy as np
import pandas as pd
import librosa

In [ ]:
# Configuration du path
BASE_PATH = "/content/drive/MyDrive/dataset"

TRAIN_PATH = os.path.join(BASE_PATH, "train.csv")
TRAIN_AUDIO_DIR = os.path.join(BASE_PATH, "train")

VALID_PATH = os.path.join(BASE_PATH, "valid.csv")
VALID_AUDIO_DIR = os.path.join(BASE_PATH, "valid")

In [ ]:
# CHARGEMENT DES DF
train_df = pd.read_csv(TRAIN_PATH)
valid_df = pd.read_csv(VALID_PATH)

train_df.info()

print("\n\n")

valid_df.info()

In [ ]:
# Sum total de la durée des audios
print("{:.2f}h d'audios dans train".format(sum(train_df['duration'])/3600))
print("{:.2f}h d'audios dans valid".format(sum(valid_df['duration'])/3600))

In [ ]:
train_df.sample(5)

In [ ]:
valid_df.sample(5)

## Graphiques pour mieux appréhender les données

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

train_df["utterance_length"] = train_df["utterance"].apply(lambda x: len(x.split()))
valid_df["utterance_length"] = valid_df["utterance"].apply(lambda x: len(x.split()))

plt.figure(figsize=(10,5))
sns.histplot(train_df["utterance_length"], bins=30, kde=False, color="blue", alpha=0.6, label="Train")
sns.histplot(valid_df["utterance_length"], bins=30, kde=False, color="orange", alpha=0.6, label="Validation")
plt.xlabel("Nombre de mots")
plt.ylabel("Nombre d'échantillons")
plt.title("Comparaison de la taille des transcriptions (Train vs Validation)")
plt.legend()
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# Histogramme des durées des audios
plt.figure(figsize=(10, 5))
plt.hist(train_df["duration"], bins=50, alpha=0.7, color="blue", label="Train", edgecolor="black")
plt.hist(valid_df["duration"], bins=50, alpha=0.7, color="orange", label="Validation", edgecolor="black")

# Titre et labels
plt.title("Comparaison de la durée des audios (Train vs Validation)")
plt.xlabel("Durée des audios (secondes)")
plt.ylabel("Nombre d'échantillons")
plt.legend()
plt.grid(axis="y", linestyle="--", alpha=0.7)

# Affichage
plt.show()


In [ ]:
print("Train - Durée moyenne:", train_df["duration"].mean(), "sec")
print("Train - Durée médiane:", train_df["duration"].median(), "sec")

print("Validation - Durée moyenne:", valid_df["duration"].mean(), "sec")
print("Validation - Durée médiane:", valid_df["duration"].median(), "sec")



> **On peut remarquer que la plupart des audios dans le dataset(train/validation)
 mis à disposition font moins de 6 secondes. Vu les contraintes en ressources de calcul (GPU T4 15Gb valable pour 4h : insuffisant pour le travail), je vais cibler uniquement les audios courts (<=5s) pour cette occasion.**



In [ ]:
# On utilise desormais une version de train_df et valid_df qui ne contienent que des audios dont la durée est inférieure ou également à 5s
train_df = train_df[train_df["duration"] <= 5]
valid_df = valid_df[valid_df["duration"] <= 5]
train_df.info()

In [ ]:
# Pretraitez le texte
import re

def pretraitement_text(text):
  text = text.lower()

  # Nettoie : espaces multiples, parenthèses et symboles arithmétiques
  text = re.sub(r"\s+", " ", text)
  text = re.sub(r'[()=+*/\\[\]]', '', text)
  text = text.strip()

  return text

train_df['utterance'] = train_df['utterance'].apply(pretraitement_text)
valid_df['utterance'] = valid_df['utterance'].apply(pretraitement_text)

train_df.head()

In [ ]:
# On ajoute le chemin complet vers les fichiers audio
def add_audio_path(df, DIR, col_name="filename"):
    df["audio"] = df[col_name].apply(lambda filename: os.path.join(DIR, filename))
    return df

train_df = add_audio_path(train_df, TRAIN_AUDIO_DIR)
valid_df = add_audio_path(valid_df, VALID_AUDIO_DIR)

train_df.head()

In [ ]:
valid_df.head()

In [ ]:
train_df["duration"].mean()

In [ ]:
if MODE_DEBUG:
  # Vérification de l'existence des fichiers audio
  def check_file_exists(row):
      """Vérifie si le fichier audio existe, en tenant compte du chemin de base."""
      file_path = row['audio']['path']
      return os.path.exists(file_path)

  print("Vérification de l'existence des fichiers audio...")

  for df in [train_df, valid_df]:
    df['files_exists'] = df.apply(check_file_exists, axis=1)
    missing_files = df[~df['files_exists']]

    if not missing_files.empty:
        print(f"Avertissement : {len(missing_files)} fichiers audio n'ont pas été trouvés.")
    else:
        print("Tous les fichiers audio sont accessibles.")

In [ ]:
if MODE_DEBUG:
  # Vérification de la cohérence des durées audio
  def verify_duration(row, tolerance=0.1):
      """
      Vérifie que la durée réelle du fichier audio correspond à celle indiquée dans le CSV.
      """
      file_path = row['audio']['path']

      try:
          # Chargement de l'audio sans rééchantillonnage
          audio, sr = librosa.load(file_path, sr=None)
          real_duration = librosa.get_duration(y=audio, sr=sr)
          expected_duration = float(row['duration'])
          return abs(real_duration - expected_duration) <= tolerance
      except Exception as e:
          print(f"Erreur lors du chargement du fichier {file_path} : {e}")
          return False

  print("Vérification de la cohérence des durées audio...")

  for df in [train_df, valid_df]:
    df['duration_match'] = df.apply(verify_duration, axis=1)

    mismatched = df[~df['duration_match']]

    if not mismatched.empty:
        print(f"Avertissement : {len(mismatched)} fichiers présentent une durée incohérente.")
    else:
        print("Les durées audio correspondent aux valeurs attendues.")

In [ ]:
# Audio lazy loading
from datasets import Dataset, Audio

train_dataset = Dataset.from_pandas(train_df)
valid_dataset = Dataset.from_pandas(valid_df)

train = train_dataset.train_test_split(test_size=0.95)["train"] # Pour éviter crash de session dû à la RAM
valid = valid_dataset.train_test_split(test_size=0.95)["train"]

train_subset = train.cast_column("audio", Audio())
valid_subset = valid.cast_column("audio", Audio())

In [ ]:
len(train_subset), len(valid_subset)

In [ ]:
# Vérification manuelle
from IPython.display import Audio, display

random_row = train_subset[9]
audio_array = random_row["audio"]["array"]

print("utterance : ", random_row["utterance"])
display(Audio(audio_array, rate=16000))



> En explorant rapidement les audios, on remarque qu'il y a des cas de traduction en français pas forcément correctes. Ceci pourrait impacter négativement les perfs du modèle vu qu'il n'avait déjà pas accès à énormenment de ressources de bonne qualité.



# Conception du modèle de SpeechTranslation

Pour construire le modèle, on utilise :
- Wav2Vec2.0 pour encoder l'audio
- Un adapter pour adapter les représentations de Wav2Vec2 au décodeur.
- Une couche de projection transforme les caractéristiques audio pour être compatibles avec le décodeur MBert.
- MBert en décodeur pour la traduction en français.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from transformers import Wav2Vec2Model, MBartForConditionalGeneration, MBartTokenizer

# class Permute(nn.Module):
#     """Module personnalisé pour permuter les dimensions d'un tensor"""
#     def __init__(self, dims):
#         super().__init__()
#         self.dims = dims

#     def forward(self, x):
#         return x.permute(*self.dims)

class SpeechTranslationModel(nn.Module):
    def __init__(self, encoder_name, decoder_name, tokenizer):
        super().__init__()
        # Encoder gelé
        self.audio_encoder = Wav2Vec2Model.from_pretrained(encoder_name)
        for param in self.audio_encoder.parameters():
            param.requires_grad = False

        # Adapter
        self.audio_adapter = nn.Sequential(
            nn.Conv1d(768, 512, kernel_size=1),  # Réduction dimensionnelle (768 -> 512)
            nn.GELU(),
            # Permute((0, 2, 1)), # Desactiver pour décharger l'empreinte mémoire
            # nn.LayerNorm(512),
            # Permute((0, 2, 1))
        )

        # Décodeur partiellement gelé
        self.decoder = MBartForConditionalGeneration.from_pretrained(decoder_name)
        for layer in self.decoder.model.decoder.layers[:8]:  # Gel des 6 premières couches
            for param in layer.parameters():
                param.requires_grad = False

        # Projection pour passer de 512 à d_model du décodeur
        self.proj = nn.Sequential(
            nn.Linear(512, self.decoder.config.d_model)
        )

        # Tokenizer
        self.tokenizer = tokenizer

    def forward(self, audio_input, audio_attention_mask, text_attention_mask, labels=None):
        # Encodage audio (Wav2Vec2 gelé)
        with torch.no_grad():
            audio_features = self.audio_encoder(audio_input, attention_mask=audio_attention_mask).last_hidden_state  # [batch, seq_len, 768]

        # Adaptation des features audio
        audio_features = audio_features.permute(0, 2, 1)     # [batch, 768, seq_len]
        audio_features = self.audio_adapter(audio_features)  # [batch, 512, seq_len]
        audio_features = audio_features.permute(0, 2, 1)     # [batch, seq_len, 512]

        # Projection pour matcher la dimension de MBart
        decoder_inputs_embeds = self.proj(audio_features)    # [batch, seq_len, d_model]

        # Ajustement de la dimension séquentielle
        target_seq_len = text_attention_mask.size(1)
        decoder_inputs_embeds = decoder_inputs_embeds.transpose(1, 2)  # [batch, d_model, seq_len]
        decoder_inputs_embeds = F.interpolate(decoder_inputs_embeds, size=target_seq_len, mode='linear', align_corners=False)
        decoder_inputs_embeds = decoder_inputs_embeds.transpose(1, 2)  # [batch, target_seq_len, d_model]

        # Décodeur MBart
        outputs = self.decoder(
            inputs_embeds=decoder_inputs_embeds,
            attention_mask=text_attention_mask,
            labels=labels,
            return_dict=True
        )

        return outputs

    def generate(self, audio_input, audio_attention_mask, target_seq_len = 150):
        # Encodage audio
        with torch.no_grad():
            audio_features = self.audio_encoder(audio_input, attention_mask=audio_attention_mask).last_hidden_state

        # Adaptation + projection
        audio_features = self.audio_adapter(audio_features.permute(0, 2, 1))
        decoder_inputs_embeds = self.proj(audio_features.permute(0, 2, 1))  # [batch, seq_len, d_model]

        decoder_inputs_embeds = decoder_inputs_embeds.transpose(1, 2)
        decoder_inputs_embeds = F.interpolate(decoder_inputs_embeds, size=target_seq_len, mode='linear', align_corners=False)
        decoder_inputs_embeds = decoder_inputs_embeds.transpose(1, 2)

        # Création d'un masque de texte rempli de 1 (pas de masquage)
        text_attention_mask = torch.ones(decoder_inputs_embeds.size()[:2], dtype=torch.long, device=decoder_inputs_embeds.device)

        # Génération avec MBart
        generated_ids = self.decoder.generate(
            inputs_embeds=decoder_inputs_embeds,
            attention_mask=text_attention_mask,
            max_length=target_seq_len,
            num_beams=5,
            early_stopping=True,
            forced_bos_token_id=self.tokenizer.lang_code_to_id["fr_XX"]
        )

        return generated_ids


In [ ]:
# Dataset personnalisé
class SpeechTranslationDataset():
    def __init__(self, dataset, processor, max_length=8000): # 1/2s
        self.dataset = dataset
        self.max_length = max_length
        self.processor = processor

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        item = self.dataset[idx]

        # Traitement audio
        audio = processor(
            item["audio"]["array"],
            sampling_rate=16000,
            return_tensors="pt"
        ).input_values.squeeze(0)  # Supprime la dimension de batch

        # Troncature/padding
        if audio.size(0) < self.max_length:
            audio = torch.nn.functional.pad(audio, (0, self.max_length - audio.size(0)))
        else:
            audio = audio[:self.max_length]

        return {
            "audio": audio,
            "text": item["utterance"]
        }

    # Collate Function
    def collate_fn(self, batch):
        # Audio processsing
        audio = [item["audio"] for item in batch]
        audio = nn.utils.rnn.pad_sequence(audio, batch_first=True)
        audio_attention_mask = (audio != 0).float()

        # Texte processing
        texts = [item["text"] for item in batch]
        max_length = getattr(tokenizer.model_max_length, "max_length", 512)
        tokenized = tokenizer(
            texts,
            max_length=max_length,
            truncation=True,
            padding="max_length",
            return_tensors="pt"
        )

        text_attention_mask = tokenized["attention_mask"]

        return {
            "audio_input": audio.to(device),
            "audio_attention_mask": audio_attention_mask.to(device),
            "labels": tokenized["input_ids"].to(device),
            "text_attention_mask": text_attention_mask.to(device)
        }

In [ ]:
# Prechargement des données
from transformers import Wav2Vec2Processor

from datasets import load_dataset
from torch.utils.data import DataLoader
from tqdm import tqdm
import sacrebleu

# Entraînement
from torch.optim.adamw import AdamW

# Boucle d'entraînement
from tqdm.auto import tqdm

# Configuration
encoder_name = "facebook/wav2vec2-base"
decoder_name = "facebook/mbart-large-cc25"

processor = Wav2Vec2Processor.from_pretrained(encoder_name)
tokenizer = MBartTokenizer.from_pretrained(decoder_name, tgt_lang="fr_XX")

train_subset_dataset = SpeechTranslationDataset(train_subset, processor)
train_loader = DataLoader(train_subset_dataset,
                          batch_size=2,
                          collate_fn=train_subset_dataset.collate_fn,
                          shuffle=True)


model = SpeechTranslationModel(encoder_name, decoder_name, tokenizer)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

optimizer = AdamW(model.parameters(), lr=5e-5)


In [ ]:
# Initalisation de wandb
import wandb

# Initialisation
wandb.login(key="")

wandb.init(project="st-FonFr",
           config={"architecture": "Wav2Vec2-MBart",
                   "dataset": "FFSTC_IWSLT2025",
                    "encoder": "wav2vec2-base",
                    "decoder": "mbart-large-cc25",
                    "lr": 5e-5,
                    "batch_size": 2
                   })

wandb.watch(model, log="all", log_freq=50)

In [ ]:
# Entrainement proprement dit
from datetime import timedelta
import time

EPOCHS = 10

losses_per_epoch = []  # loss moyenne par epoch
perplexities = []      # perplexité par epoch
time_per_epoch = []    # Temps d'exécution par epoch

start_time = time.time()
total_steps = len(train_loader) * EPOCHS

def calculate_perplexity(loss):
    return torch.exp(torch.tensor(loss)) # exp(loss)

# Boucle d'entrainement
for epoch in range(EPOCHS):
    model.train()

    total_loss = 0
    step = 0

    epoch_start_time = time.time()
    print(f" Epoch : {epoch+1}/{EPOCHS}")

    for batch_idx, batch in enumerate(train_loader):
        current_step = epoch * len(train_loader) + batch_idx

        # Forward + backward
        outputs = model(
            batch["audio_input"].to(device),
            audio_attention_mask=batch["audio_attention_mask"].to(device),
            labels=batch["labels"].to(device),
            text_attention_mask=batch["text_attention_mask"].to(device)
        )

        loss = outputs.loss
        loss.backward()

        # Mise à jour des poids tous les 5 steps (economie de  GPU)
        if (batch_idx + 1) % 5 == 0:
            optimizer.step()
            optimizer.zero_grad()

        # Calcul des métriques
        total_loss += loss.item()
        avg_loss = total_loss / (batch_idx + 1)
        perplexity = calculate_perplexity(avg_loss).item()

        # Calcul du temps écoulé et restant
        elapsed = time.time() - start_time
        steps_per_sec = elapsed / (current_step + 1)
        remaining = steps_per_sec * (total_steps - current_step - 1)

        # Affichage tous les 10 steps
        if (batch_idx + 1) % 10 == 0:
            print(f"Epoch {epoch+1}/{EPOCHS} | Step {batch_idx+1}/{len(train_loader)} | "
                  f"Loss: {loss.item():.4f} | Avg Loss: {avg_loss:.4f} | "
                  f"Perplexity: {perplexity:.2f} | "
                  f"Elapsed: {str(timedelta(seconds=int(elapsed)))} | "
                  f"Remaining: ~{str(timedelta(seconds=int(remaining)))}",
                  end='\r')

        wandb.log({"train/loss": loss.item(),
                   "train/avg_loss": avg_loss,
                   "train/perplexity": perplexity,
                   "lr": optimizer.param_groups[0]['lr']},
                   step=step + epoch * len(train_loader))

    epoch_time = time.time() - epoch_start_time
    time_per_epoch.append(epoch_time)
    losses_per_epoch.append(avg_loss)
    perplexities.append(perplexity)

    print(f"\nEpoch {epoch+1} completed - Avg Loss: {avg_loss:.4f} | Perplexity: {perplexity:.2f} | Time: {epoch_time:.2f} sec")

print(f"\nTraining completed in {str(timedelta(seconds=int(time.time() - start_time)))}")

In [ ]:
# Analyse
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(range(1, EPOCHS + 1), losses_per_epoch, marker='o', linestyle='-', color='b')
plt.xlabel("Epochs")
plt.ylabel("Average Loss")
plt.title("Loss Evolution")
plt.grid()

plt.subplot(1, 2, 2)
plt.plot(range(1, EPOCHS + 1), perplexities, marker='s', linestyle='-', color='r')
plt.xlabel("Epochs")
plt.ylabel("Perplexity")
plt.title("Perplexity Evolution")
plt.grid()

plt.show()


> On remarque que la courbe de perte du modèle dimunie progressivement au fil des epochs, on peut donc dire que

In [ ]:
from sacrebleu import corpus_bleu
from tqdm import tqdm
from torch.utils.data import DataLoader

def evaluate(model, dataset):
    model.eval()
    references = []  # Format: [ [ref1], [ref2], ... ]
    hypotheses = []

    dataloader = DataLoader(dataset, batch_size=2, collate_fn=dataset.collate_fn)

    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Evaluation"):
            # Génération
            generated_ids = model.generate(
                batch["audio_input"],
                batch["audio_attention_mask"]
            )

            # Décodage des prédictions
            preds = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
            hypotheses.extend(preds)

            # Décodage des références (à partir des input_ids)
            refs = tokenizer.batch_decode(batch["labels"], skip_special_tokens=True)
            references.extend([[r] for r in refs])  # Format SacreBLEU

    bleu = corpus_bleu(hypotheses, references)
    return {"bleu": bleu.score, "predictions": hypotheses, "references": references}

# Utilisation
valid_dataset = SpeechTranslationDataset(valid_subset, processor)
results = evaluate(model, valid_dataset)
print(f"BLEU Score: {results['bleu']}")

In [ ]:
import torch
import torchaudio
from transformers import Wav2Vec2Processor

def prepare_audio_input(audio_path, processor, max_length=8000, device="cuda" if torch.cuda.is_available() else "cpu"):
    """
    Transforme un fichier audio en entrée compatible avec votre modèle.

    Args:
        audio_path (str): Chemin vers le fichier audio (.wav)
        processor (Wav2Vec2Processor): Processor pour la normalisation audio
        max_length (int): Longueur maximale en échantillons (16kHz = 1 seconde)
        device (str): Device pour les tensors (cpu/cuda)

    Returns:
        dict: {"audio_input": tensor, "audio_attention_mask": tensor}
    """
    # Chargement et resampling si nécessaire
    waveform, sample_rate = torchaudio.load(audio_path)
    if sample_rate != 16000:
        waveform = torchaudio.functional.resample(waveform, sample_rate, 16000)

    # Normalisation avec le processor (identique au dataset)
    audio = processor(
        waveform.squeeze().numpy(),  # Convertir en numpy array
        sampling_rate=16000,
        return_tensors="pt"
    ).input_values.squeeze(0)

    # Troncature/Padding
    if audio.size(0) < max_length:
        audio = torch.nn.functional.pad(audio, (0, max_length - audio.size(0)))
    else:
        audio = audio[:max_length]

    # Création du masque d'attention
    audio_attention_mask = (audio != 0).float()

    return {
        "audio_input": audio.unsqueeze(0).to(device),  # Ajoute une dimension batch [1, seq_len]
        "audio_attention_mask": audio_attention_mask.unsqueeze(0).to(device)
    }


In [ ]:
from transformers import Wav2Vec2Processor

# processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base")

audio_input = prepare_audio_input("/kaggle/input/ffstc-2025/train/19404.wav", processor)

# Génération avec le modèle
with torch.no_grad():
    generated_ids = model.generate(
        audio_input=audio_input["audio_input"],
        audio_attention_mask=audio_input["audio_attention_mask"],
    )

translation = tokenizer.decode(generated_ids[0], skip_special_tokens=True)

print(translation)

# Save Model

In [ ]:
# Save model
import torch
from pathlib import Path
from datetime import datetime
import json

def save_full_model(model, processor, tokenizer, save_dir="st-fonfr"):
    """Sauvegarde complète pour production"""
    save_path = Path(save_dir)
    save_path.mkdir(exist_ok=True)

    # State dict
    torch.save(model.state_dict(), save_path / "model_weights.pth")

    # Processor & Tokenizer
    processor.save_pretrained(save_path)
    tokenizer.save_pretrained(save_path)

    # 3. Metadata minimale
    metadata = {
        "model_class": model.__class__.__name__,
        "timestamp": datetime.now().isoformat(),
        "audio_sample_rate": processor.feature_extractor.sampling_rate,
        "target_language": tokenizer.tgt_lang
    }

    with open(save_path / "metadata.json", "w") as f:
        json.dump(metadata, f)

def load_full_model(save_dir="st-fonfr", device="cuda"):
    save_path = Path(save_dir)

    # Chargement processor/tokenizer
    processor = Wav2Vec2Processor.from_pretrained(save_path)
    tokenizer = MBartTokenizer.from_pretrained(save_path)

    # Reconstruction du modèle vide
    model = SpeechTranslationModel(processor, tokenizer).to(device)

    # Chargement des poids
    model.load_state_dict(torch.load(save_path / "model_weights.pth", map_location=device))

    return model, processor, tokenizer

In [ ]:
# SAUVEGARDE DU MODELE EN LOCAL
save_full_model(model, processor, tokenizer)

# Upload vers HuggingFace

In [ ]:
%%capture
!pip -s install huggingface_hub

In [ ]:
!huggingface-cli login

In [ ]:
from huggingface_hub import upload_folder

local_model_path = "st-FonFr"
repo_id = "TitanSage02/st-wav2vec2-mbart-fon-fr"

upload_folder(
    repo_id=repo_id,
    folder_path=local_model_path,
    commit_message="Upload du modèle Speech Translation Fon-Fr"
)

print(f"Modèle uploadé avec succès vers le Hugging Face Hub : https://huggingface.co/{repo_id}")